# Домашнее задание HW 4

## Задание HW 4

Домашнее задание состоит из нескольких блоков.


### Эксперименты в ipynb ноутбуках (11 баллов)
- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(5 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(2 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(2 балла)**
- Придумать как можно обработать рекомендации для холодных пользователей. **(2 балла)**

Примечание: за невоспроизводимый код в ноутбуках (например, нарушен порядок выполнения ячеек, вызываются переменные, которые нигде не были объявлены ранее и.т.п) будут штрафы на усмотрение проверяющего.


### Реализация итоговой модели в сервисе (9 баллов)
- Пробитие бейзлайна $MAP@10 \geq 0.074921$ **(6 баллов)**
- Код сервиса соответствует критериям читаемости и воспроизводимости **(3 балла)**

## Imports

In [ ]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
from collections import Counter
import time
import numpy as np
from pprint import pprint
import dill
from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
import nmslib

import optuna

from pathlib import Path

from lightfm import LightFM

## Подгружаем данные

In [ ]:
DATA_PATH = Path("data_original")

In [ ]:
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

In [ ]:
users

In [ ]:
items

In [ ]:
interactions

## Подготовка данных

In [ ]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [ ]:
max_date = interactions[Columns.Datetime].max()

In [ ]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [ ]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

In [ ]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [ ]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [ ]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

## Подготовка фичей

### Пользовательские фичи

In [ ]:
# Сколько в % незаполненных полей у Пользователя
users.isnull().sum()/len(users)

In [ ]:
# % не большой => можно заменить пустые значения
users.fillna('Unknown', inplace=True)

In [ ]:
# оставляем у df users только тех, кто попал в train
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [ ]:
users

In [ ]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

### Фичи фильмов

In [ ]:
# Сколько в % незаполненных полей у Item
items.isnull().sum()/len(items)

In [ ]:
# оставляем у df users только тех, кто попал в train
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

### Жанр

In [ ]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

### Содержание

In [ ]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

### Режисер

In [ ]:
items["director"] = items["directors"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
director_feature = items[["item_id", "director"]].explode("director")
director_feature.columns = ["id", "value"]
director_feature["feature"] = "director"
director_feature.head()

### Страна

In [ ]:
items["country"] = items["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = items[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

### Год Выпуска

In [ ]:
year_feature = items.reindex(columns=[Columns.Item, "release_year"])
year_feature.columns = ["id", "value"]
year_feature["feature"] = "release_year"
year_feature.head()

In [ ]:
items

In [ ]:
# Объединяем фичи
item_features = pd.concat((genre_feature, content_feature, country_feature, year_feature, director_feature))
item_features

## Метрики

In [ ]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)


pprint(metrics)

## Подбор гиперпараметртов

In [ ]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "director", "country", "release_year"],
)

TEST_USERS = test[Columns.User].unique()

In [ ]:
results_opto = []
def objective(trial):
    # общие параметры
    K_RECOS = 10
    RANDOM_STATE = 42
    NUM_THREADS = 4
    N_FACTORS = trial.suggest_int("factors", 32, 64, 32, log=False)

    reco_model = trial.suggest_categorical("reco_model", ["ALS", "LightFM"])
    model_quality_opto = {"model": f"{reco_model}_{trial.number}"}

    if reco_model == "ALS":
        # гиперпараметры для ALS
        regularization = trial.suggest_float("regularization",
                                             1e-4,
                                             1.0,
                                             log=False)
        iterations = trial.suggest_int("iterations", 1, 6, 2, log=False)
        is_fitting_features = True
        # Инициализация ALS
        model = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(factors=N_FACTORS,
                                 random_state=RANDOM_STATE,
                                 num_threads=NUM_THREADS,
                                 regularization=regularization,
                                 iterations=iterations),
            fit_features_together=is_fitting_features,
        )
    else:
        # гиперпараметры для Lightfm
        N_EPOCHS = trial.suggest_int("epochs", 1, 50, 10, log=False)  # Lightfm
        USER_ALPHA = trial.suggest_float("user_alpha",
                                         1e-3,
                                         1.0,
                                         log=True)  # Lightfm
        ITEM_ALPHA = trial.suggest_float("item_alpha", 1e-3, 1.0,
                                         log=True)  # Lightfm
        LEARNING_RATE = trial.suggest_float("learning_rate",
                                            1e-4,
                                            1.0,
                                            log=True)  # Lightfm
        # Инициализация Lightfm
        model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss=loss,
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

    # обучение модели
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )

    # Подсчет метрик
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality_opto.update(metric_values)
    results_opto.append(model_quality_opto)

    return metric_values.get('MAP@10') # максимизируемая метрика

In [ ]:
# запуск подбора гиперпараметров
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

pprint(f"Number of finished trials: {len(study.trials)}")
trial = study.best_trial
pprint(f"Best trial: {trial}")

## Approximate Nearest Neighbors

In [ ]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4
N_FACTORS = 64
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.03 # Lightfm

In [ ]:
model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss='warp',
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

model.fit(dataset)

In [ ]:
recos = model.recommend(
    users=TEST_USERS,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [ ]:
results = []
model_quality = {'model': 'LightFM'}
metric_values = calc_metrics(metrics, recos, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [ ]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
dill_file = Path().cwd().parent / 'service' / 'models'

with open(dill_file / 'LightFM_0.077731.dill', 'wb') as f:
    dill.dump(model, f)

In [ ]:
dill_file = Path().cwd().parent / 'service' / 'data'

with open(dill_file / 'dataset_LightFM_0.077731.dill', 'wb') as f:
    dill.dump(dataset, f)

In [ ]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [ ]:
user_embeddings.shape, item_embeddings.shape

In [ ]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()

    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [ ]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

In [ ]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

In [ ]:
user_id = 30
user_embeddings[user_id]

In [ ]:
augmented_user_embeddings[user_id]

In [ ]:
item_id = 0
item_embeddings[item_id]

In [ ]:
augmented_item_embeddings[item_id]

In [ ]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

# Number of neighbors
K=10

# Space name should correspond to the space name
# used for brute-force search
space_name='negdotprod'

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

In [ ]:

# Intitialize the library, specify the space, the type of the vector and add data points
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR)
index.addDataPointBatch(augmented_item_embeddings)

In [ ]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params)
end = time.time()
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

In [ ]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

In [ ]:
user_index = np.where(dataset.user_id_map.external_ids == 1032142)[0][0]
query_matrix = augmented_user_embeddings[user_index]

In [ ]:
query_matrix

In [ ]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time()
nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)
end = time.time()
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' %
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

In [ ]:
nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)

In [ ]:
nbrs

In [ ]:
dataset.user_id_map.external_ids[nbrs[0][0]]

In [ ]:
user_id = 6064
K_RECOS = 10
N = 50
cnt = Counter(train['item_id'].values)
idf = pd.DataFrame.from_dict(cnt, orient='index', columns=['doc_freq']).reset_index()
n = train.shape[0]
idf['idf'] = idf['doc_freq'].apply(lambda x: np.log((1 + n) / (1 + x) + 1))

users_inv_mapping = dict(enumerate(train['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}


items_inv_mapping = dict(enumerate(train['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}


def generate_implicit_recs_mapper(model, N, users_mapping, users_inv_mapping):
    def _recs_mapper(user):
        user_id = users_mapping[user]
        recs = model.similar_items(user_id, N=N)
        return [users_inv_mapping[user] for user, _ in recs], [sim for _, sim in recs]
    return _recs_mapper

In [ ]:
mapper = generate_implicit_recs_mapper(
    userknn,
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [ ]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

In [ ]:
wwatched = {k:v['item_id'] for k,v in json.loads(watched.T.to_json()).items()}

In [ ]:
def recommend_nmslib(user_id, k_recos):
    recss = {}
    user_index = np.where(dataset.user_id_map.external_ids == user_id)
    if len(user_index[0]):
        user_index = user_index[0][0]
        query_matrix = augmented_user_embeddings[user_index]
        nbrs = index.knnQueryBatch([query_matrix],
                                   k=K,
                                   num_threads=num_threads)
        recss['similar_user_id'], recss[
            'similarity'] = dataset.user_id_map.external_ids[nbrs[0][0]], nbrs[0][1]
        recss['item_id'] = [
            wwatched.get(f"{x}") for x in recss['similar_user_id']
        ]
        recs = pd.DataFrame(recss)
        recs = recs.explode('item_id')
        recs = recs.sort_values(['similarity'], ascending=False)
        recs = recs.merge(idf[['index', 'idf']],
                          left_on='item_id',
                          right_on='index',
                          how='left').drop(['index'], axis=1)
        recs['rank_idf'] = recs['similarity'] * recs['idf']
        recs = recs.sort_values(['rank_idf'], ascending=False)
        recs.dropna(inplace=True)
        return recs['item_id'].unique()[:k_recos]
    else:
        return None

In [ ]:
res = []
for user in TEST_USERS[:]:
    user_index = np.where(dataset.user_id_map.external_ids == user)
    if len(user_index[0]):
        labels = recommend_nmslib(user, k_recos=10)
        labels_u = np.zeros((len(labels), 3), dtype=int)
        labels_u[:, 0] = user

        # проставляем порядок треков
        labels_u[:, 2] = range(len(labels))
        labels_u[:, 1] = labels
    #     labels_u[:, 2] = distances
        res.extend(labels_u)

In [ ]:
recos_ = pd.DataFrame(res, columns=['user_id', 'item_id', 'rank'])

In [ ]:
# подсчет метрик
results = []
model_quality = {'model': 'nmslib'}
metric_values = calc_metrics(metrics, recos_, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [ ]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
def recommend_all(query_factors, index_factors, topn=10):
    output = query_factors.dot(index_factors.T)
    argpartition_indices = np.argpartition(output, -topn)[:, -topn:]

    x_indices = np.repeat(np.arange(output.shape[0]), topn)
    y_indices = argpartition_indices.flatten()
    top_value = output[x_indices, y_indices].reshape(output.shape[0], topn)
    top_indices = np.argsort(top_value)[:, ::-1]

    y_indices = top_indices.flatten()
    top_indices = argpartition_indices[x_indices, y_indices]
    labels = top_indices.reshape(-1, topn)
    distances = output[x_indices, top_indices].reshape(-1, topn)
    return labels, distances
